In [ ]:
%matplotlib inline
from pyleecan.Functions.save import save
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
from util.toyota_prius_generator import Toyota_Prius_Generator

from pyleecan.Classes.MachineIPMSM import MachineIPMSM
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.InputCurrent import InputCurrent
import matplotlib.pyplot as plt
from numpy import sqrt
from os import makedirs
from os.path import join
# Load the machine
from util.simulation import *
from util.failures import *
from IPython.display import display, HTML


In [ ]:
# Dossier de sauvegarde des machines
save_dir = "machines_custom"
makedirs(save_dir, exist_ok=True)

In [ ]:
# Machine saine (Ntcoil = 150)
gen_ok = Toyota_Prius_Generator(Ntcoil=150)
machine_ok = MachineIPMSM(
    name="Toyota_Prius_Ntcoil_150",
    shaft=gen_ok.create_shaft(),
    rotor=gen_ok.create_rotor(),
    stator=gen_ok.create_stator()
)
save(machine_ok, join(save_dir, machine_ok.name + ".json"))

In [ ]:
#  Machine défectueuse (Ntcoil = 2)
gen_defect = Toyota_Prius_Generator(Ntcoil=2)
machine_def = MachineIPMSM(
    name="Toyota_Prius_Ntcoil_2",
    shaft=gen_defect.create_shaft(),
    rotor=gen_defect.create_rotor(),
    stator=gen_defect.create_stator()
)
save(machine_def, join(save_dir, machine_def.name + ".json"))

print(" Machines générées et sauvegardées dans :", save_dir)

In [ ]:
%matplotlib inline

# Load the machine
IPMSM_A = load("machines_custom/Toyota_Prius_Ntcoil_2.json")
# In Jupyter notebook, we set is_show_fig=False to skip call to fig.show() to avoid a warning message
# All plot methods return the corresponding matplotlib figure and axis to further edit the resulting plot
fig, ax = IPMSM_A.plot(is_show_fig=False)
plt.show()

In [ ]:
print(IPMSM_A.stator.winding.wind_mat)

In [ ]:
%matplotlib inline

# Load the machine
IPMSM_B = load("machines_custom/Toyota_Prius_Ntcoil_150.json")
# In Jupyter notebook, we set is_show_fig=False to skip call to fig.show() to avoid a warning message
# All plot methods return the corresponding matplotlib figure and axis to further edit the resulting plot
fig, ax = IPMSM_A.plot(is_show_fig=False)
plt.show()

In [ ]:
print(IPMSM_B.stator.winding.wind_mat)

In [ ]:
# Chargement des machines
machine_ref = load(join(DATA_DIR, "Machine", "Toyota_Prius.json"))
machine_ntcoil = load("machines_custom/Toyota_Prius_Ntcoil_150.json")
machine_defect = load("machines_custom/Toyota_Prius_Ntcoil_2.json")

In [ ]:
simu_ref = load_simulation(name="ref", machine=machine_ref, rotor_speed=3000, stop=1, num_steps=4000)
simu_defect = load_simulation(name="def", machine=machine_defect, rotor_speed=3000, stop=1, num_steps=4000)
simu_ntcoil = load_simulation(name="ntcoil", machine=machine_ntcoil, rotor_speed=3000, stop=1, num_steps=4000)

In [ ]:
out1 = simu_ref.run()
out2 = simu_defect.run()
out3 = simu_ntcoil.run()

compare_simulation_results(
    out1=out_ref,
    out2=out_defect,
    legend_list=["Toyota_Prius_Ref", "Toyota_Prius_Ntcoil_2"]
)

compare_simulation_results(
    out1=out_ref,
    out2=out_ntcoil,
    legend_list=["Toyota_Prius_Ref", "Toyota_Prius_Ntcoil_150"]
)

In [ ]:
out1.save(save_path='machines_custom/compar_simule', is_folder=True, type_handle_old=2, type_compression=0)

par cette comparaison de  trois configurations de la machine Toyota Prius, la version originale, 
avec une version optimisée avec un nombre de spires par demi-bobine augmenté à 150 (Ntcoil = 150), 
et une version défectueuse avec seulement 2 spires (Ntcoil = 2).

Les résultats montrent que la machine défectueuse présente une forte dégradation du flux magnétique et du couple électromagnétique, traduisant une perte significative de performance. La version optimisée, quant à elle, génère un flux plus intense et un couple nettement plus élevé que la référence. Les spectres fréquentiels révèlent également des différences claires en amplitude selon la configuration. Ainsi, le paramètre Ntcoil influence fortement la qualité du fonctionnement électromagnétique.